<a href="https://colab.research.google.com/github/lizardnote/Text-Analytics/blob/main/BM25_algorithm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# BM25 알고리즘
사용자가 특정 단어를 검색했을 때, 어떤 문서가 더 적합한지 문서의 순위를 결정하는 알고리즘

**단어 빈도**  
사용자가「인공지능」이라는 단어를 검색했다고 하자.

 문서가 두 개 있는데, 첫 번째 문서에는 「인공지능」이 여러 번 반복해서 등장하고, 두 번째 문서에는 딱 한 번만 등장했을때 첫 번째 문서가 더 관련이 깊다고 생각하게 된다.

 BM25 알고리즘 역시 이러한 판단을 하며 이것을 **단어 빈도(Term Frequency, TF)**라고 합니다. 특정 단어가 문서 내에 더 자주 나타날수록 해당 문서에서 그 단어는 중요한 역할을 한다고 판단한다.

**역문서 빈도**  
만약 모든 문서에 너무 흔하게 나타나는 「그리고」, 「또한」 같은 단어라면 어떨까?

이런 단어들은 아무리 문서 내에서 빈도가 높아도 특정 문서의 중요도를 판단하는 데 도움이 되지 않는다.

반면 「인공지능」, 「머신러닝」, 「딥러닝」과 같은 특정 주제를 나타내는 단어들은 모든 문서에 흔히 등장하지 않고, 일부 문서에만 특별히 등장한다. 이런 희귀한 단어들은 문서의 주제를 보다 명확하게 드러내는 단서가 되므로, BM25는 이런 단어에 더 높은 점수를 부여한다. 이것이 바로 BM25가 중요하게 고려하는 두 번째 요소, **역문서 빈도(Inverse Document Frequency, IDF)**이다.

역문서 빈도(IDF)를 조금 더 쉽게 표현하면, 특정 단어가 전체 문서 집합에서 얼마나 「희귀한지」를 수치화한 것이고 IDF의 공식은 다음과 같다.

$$
\text{IDF} = \log\frac{N - n + 0.5}{n + 0.5}
$$

각 항목을 쉽게 설명하면 다음과 같다.

- $N$ : 전체 문서의 개수입니다.
- $n$ : 특정 단어가 나타나는 문서의 개수입니다.

즉, 특정 단어가 전체 문서 중에서 적은 수의 문서에서만 나타날수록, 더 희귀한 단어일수록 IDF 값은 높아지고, 이 단어가 해당 문서에 등장할 때의 중요도를 더 높게 판단하게 된다.

**문서 길이**  
마지막으로, BM25는 문서 길이라는 요소까지 고려하여 점수를 조정한다. 같은 횟수로 등장한 단어라도 문서 길이에 따라 그 중요성이 달라지기 때문이다. 예를 들어, 5000개의 단어로 이루어진 긴 문서에 「인공지능」이 5번 등장한 것과, 500개의 단어로 이루어진 짧은 문서에 같은 「인공지능」이 5번 등장한 것은 의미가 다르다. 긴 문서에서는 같은 횟수라도 그 중요성이 상대적으로 떨어질 수밖에 없습니다. BM25는 문서 길이를 기준으로, 긴 문서에 등장한 빈도의 가치를 낮추고, 짧은 문서에 등장한 빈도의 가치를 높이는 보정을 진행한다.

**세가지 요소를 결합한 공식**

지금까지 설명한 세 가지 요소, 즉  
① 단어의 빈도(TF),  
② 단어가 얼마나 희귀한지(IDF),  
③ 문서의 길이  
를 모두 반영한 최종적인 BM25 공식은 다음과 같다.

$$
\text{BM25 점수} = \text{IDF} \times \frac{f \times (k_1 + 1)}{f + k_1 \times \left(1 - b + b \times \frac{L}{\text{avgL}}\right)}
$$

각 항목의 의미를 다시 간단히 정리하면 다음과 같다.

- $f$ : 특정 단어가 문서에서 등장한 빈도(횟수)
- $L$ : 문서의 길이(문서가 가진 전체 단어 수)
- $avgL$ : 문서 전체의 평균 길이
- $k_1$, $b$ : 빈도와 문서 길이를 조정하는 상수 (일반적으로 $k_1 = 1.2 \sim 2.0$, $b = 0.75$ 정도로 사용)
- IDF : 앞서 설명한 단어의 희귀성 지표(역문서 빈도)

정리하면, BM25는 사용자가 검색한 「인공지능」이라는 단어가 특정 문서 내에서 많이 등장할수록(빈도가 높을수록), 전체 문서 중 매우 적은 수의 문서에만 나타나는 희귀한 단어일수록(IDF가 클수록), 그리고 문서의 길이가 짧을수록(L이 avgL보다 작을수록), 더 높은 점수를 주어 해당 문서를 사용자가 원하는 검색 결과의 상위에 배치한다.

In [1]:
!pip -q install langchain openai tiktoken langchain-community rank_bm25 sentence-transformers chromadb httpx==0.27.2 pypdf

In [2]:
import tiktoken
import openai
from langchain.embeddings.sentence_transformer import SentenceTransformerEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.document_loaders import TextLoader
from langchain.document_loaders import PyPDFLoader
from langchain.document_loaders import WebBaseLoader
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.retrievers  import BM25Retriever
from langchain.retrievers import EnsembleRetriever
from langchain_community.vectorstores import FAISS

In [3]:
!wget https://wdr.ubion.co.kr/wowpass/img/event/gsat_170823/gsat_170823.pdf

--2025-05-29 07:27:01--  https://wdr.ubion.co.kr/wowpass/img/event/gsat_170823/gsat_170823.pdf
Resolving wdr.ubion.co.kr (wdr.ubion.co.kr)... 61.100.182.43
Connecting to wdr.ubion.co.kr (wdr.ubion.co.kr)|61.100.182.43|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1253369 (1.2M) [application/pdf]
Saving to: ‘gsat_170823.pdf.1’

gsat_170823.pdf.1   100%[===================>]   1.19M   474KB/s    in 2.6s    

2025-05-29 07:27:05 (474 KB/s) - ‘gsat_170823.pdf.1’ saved [1253369/1253369]



In [4]:
## pdf 파일로드 하고 쪼개기
loader = PyPDFLoader('https://wdr.ubion.co.kr/wowpass/img/event/gsat_170823/gsat_170823.pdf')
pages = loader.load_and_split()
print(len(pages))

27


In [5]:
model_huggingface = HuggingFaceEmbeddings(model_name='BAAI/bge-m3')

<ipython-input-5-029e6670ad57>:1: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  model_huggingface = HuggingFaceEmbeddings(model_name='BAAI/bge-m3')
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.w

Document(metadata={이 파일의 정보}, page_content='내용')

In [6]:
pages[0].page_content

'2\n01 삼성전자 기업분석\n(Samsung Electronics Co., Ltd)\nⅠ 기업 일반 \n1  기업개요\n1) 기업소개 \n본사주소 경기도 수원시 영통구 삼성로 129(매탄동 416)\n사업분야 삼성그룹의 대표 기업으로 휴대폰, 정보통신기기, 반도체, TV 등을 생산 판매하는 제조업체\n홈페이지 www.samsung.com/sec 구분 전기전자 대기업  \n설립일 1961년 07월 01일 대표이사 권오현 \n총자산1) 244조 매출액2) 200조\n임직원수 95,374명 \n∙ 1975년 1월 주식시장 상장\n∙ 1984년 2월 삼성전자공업주식회사->삼성전자주식회사로 사명 변경 \n∙ CE(Consumer Electronics), IM(Information technology & Mobile communications), DS(Device Solutions) \n3개의 부문으로 나누어 독립 경영.\n부문 제품\nCE TV, 모니터, 냉장고, 세탁기, 에어컨, 프린터, 의료기기 등\nIM HHP, 네트워크시스템, 컴퓨터, 디지털카메라 등\nDS DRAM, NAND Flash, 모바일AP, LCD패널, OLED패널, LED 등 \n∙ 주요 사업은 전자전지기계 등 제조, 전자통신기 등 제도, 컴퓨터 등 제조, 반도체 제조·조립 등. 주요 \n제품과 구성비율은 HHP, 네트워크시스템, 컴퓨터, 디지털카메라 등 54%, TV, 모니터, 냉장고, 세탁기, \n에어컨 등 2 4 % ,  D R A M ,  N A N D  F l a s h ,  모바일 AP 등 19% 등으로 구분 \n∙ 지역별로는 본사를 거점으로 한국 및 CE, IM 부문 산하 해외 9개 지역총괄과 DS 부문 산하 해외 5개 \n지역총괄의 생산ㆍ판매법인 등 165개의 동종업종을 영위하는 종속기업으로 구성\n∙ 국내사업장 : 수원, 구미, 기흥, 화성, 온양, 광주\n∙ 해외사업장 : 86개국 220개지점 (15년 말 기준)\n1) 연결재무재표 기준\n2) 연결재무재표 기준'

### chunking
매우 긴 문서를 적절한 검색 단위로 나누는 것
- 길이단위 chunking
- 시맨틱 chunking
  - LLM based chunker
  - Fine-tuned model based chunker
  - 합성 데이터 생성 방식

In [7]:
## chunk로 쪼개기
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=0)
texts = text_splitter.split_documents(pages)

In [8]:
len(texts)

69

In [9]:
len(texts[0].page_content)

490

# BM-25

In [10]:
bm25_retriever = BM25Retriever.from_documents(texts)
bm25_retriever.k = 2

In [11]:
# BM25 리트리버만 사용하는 경우
docs = bm25_retriever.get_relevant_documents("삼성전자의 사업 영역은?")

<ipython-input-11-6a79492056c2>:2: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  docs = bm25_retriever.get_relevant_documents("삼성전자의 사업 영역은?")


In [12]:
len(docs)

2

In [13]:
for i in docs:
  print(i.metadata)
  print(":")
  print(i.page_content.replace('\n',' '))
  print(len(i.page_content.replace('\n',' ')))
  print("*"*30)

{'producer': 'itext-paulo-155 (itextpdf.sf.net-lowagie.com)', 'creator': 'nPDF (pdftk 1.41)', 'creationdate': '2017-08-16T00:21:02-08:00', 'moddate': '2017-08-16T00:21:02-08:00', 'source': 'https://wdr.ubion.co.kr/wowpass/img/event/gsat_170823/gsat_170823.pdf', 'total_pages': 27, 'page': 10, 'page_label': '11'}
:
12 Q2 삼성전자의 Harman사 인수에 대해 어떻게 생각하십니까 ? A  삼성전자가 최근 80억달러(약 9조)에 미국의 전장업체(자동차 전자기기에 대한 사업)인 Harman 사를  인수하기로 결정했습니다. 하만사는 세계적으로 자동차용 인포테인먼트와 텔레매틱스 시장에서 점유 율 1~2위를 달리고 있으며 매출도 상당합니다. 이번 삼성전자의 하만 인수는 모바일 및 가전 시장에서  나아가 Connected Car 시장 진입에 진출했다는 것에 큰 의의가 있습니다. 향후 10년 시장 내 Connected  Car의 비중은 90%에 육박할 전망입니다. LG전자 역시 자동차부품(VC) 사업 부문에 있어 상당한 경쟁  위협 요인가 될 것으로 내다봤습니다. <관련기사> 전장사업 강자 하만과 손잡은 삼성…스마트카 ‘티어1’ 노린다 (한국경제. 2016-11-21) 삼성전자는 자동차 전장(電裝)사업에서 후발 주자다. 2005년 전장사업을 시작한 LG전자는
474
******************************
{'producer': 'itext-paulo-155 (itextpdf.sf.net-lowagie.com)', 'creator': 'nPDF (pdftk 1.41)', 'creationdate': '2017-08-16T00:21:02-08:00', 'moddate': '2017-08-16T00:21:02-08:0

# 임베딩
- 검색 대상의 문서들을 임베딩 한 후 적재하는 곳: 벡터 데이터베이스
- 랭체인을 통해 우리가 사용할 수 있는 대표적인 벡터 데이터베이스 : 크로마, 파이스

In [14]:
#벡터 DB의 임베딩으로는 오픈소스 임베딩 사용
chroma_vector = Chroma.from_documents(texts, model_huggingface)
chroma_retriever = chroma_vector.as_retriever(search_kwargs={'k':2})

In [15]:
# 크로마 리트리버만 사용하는 경우
docs = chroma_retriever.invoke("삼성전자의 사업 영역은?")
for i in docs:
    print(i.metadata)
    print(":")
    print(i.page_content.replace('\n',' '))
    print(len(i.page_content.replace('\n',' ')))
    print("*"*30)

{'creator': 'nPDF (pdftk 1.41)', 'producer': 'itext-paulo-155 (itextpdf.sf.net-lowagie.com)', 'page_label': '10', 'page': 9, 'creationdate': '2017-08-16T00:21:02-08:00', 'source': 'https://wdr.ubion.co.kr/wowpass/img/event/gsat_170823/gsat_170823.pdf', 'moddate': '2017-08-16T00:21:02-08:00', 'total_pages': 27}
:
11 Ⅱ 기업 상세 분석 1  사업분야(내용) Q1 삼성전자의 대표적 사업분야에 대해 설명할 수 있습니까 ? A  삼성전자는 크게 CE(Consumer Electronics) 사업부문, IM(Information technology & Mobile communica- tion) 사업부문, DS(Device Solutions) 사업부문 등 3개 사업부문으로 나누어 독립 경영을 합니다. ⑴ Consumer Electronics (CE) 부문  ① 영상디스플레이 : 진화하는 스마트TV, 초대형 프리미엄 TV 전략으로 8년 연속 세계 1위에  도전 ② 생활가전 : 새로운 기술과 가치 창출로 생활과 문화를 바꾸는 혁신을 준비 ③ 의료기기 : 정확하고 빠른 진단을 도와주는 새롭고 혁신적인 의료기기를 개발 ⑵ Information technology & Mobile communication (IM) 부문 ① 무선 : 인간 중심의 혁신으로 소비자들이 열망하는 새로운 가치와 편의를 지속적으로 제공
499
******************************
{'creationdate': '2017-08-16T00:21:02-08:00', 'total_pages': 27, 'moddate': '2017-08-16T00:21:02-08:00', 'producer': 'itext-paulo-155 (itextpdf.sf.net-

# 앙상블
## EnsenbleRetriever 작동 과정 (상세 예시)
먼저 두 검색기가 있고, 각각 k=4개의 문서를 반환한다고 가정해보자

#### 1. 개별 검색기 결과 (각 검색기의 상위 4개 문서)
```
BM25 검색기 결과:
- 문서 A: 점수 0.9
- 문서 B: 점수 0.8
- 문서 C: 점수 0.7
- 문서 D: 점수 0.6

임베딩 검색기 결과:
- 문서 A: 점수 0.95 (BM25와 중복)
- 문서 E: 점수 0.85
- 문서 F: 점수 0.75
- 문서 B: 점수 0.65 (BM25와 중복)
```
여기서 문서 A와 B는 두 검색기 모두에서 반환되었습니다.[링크 텍스트](https://)

#### 2. 가중치 적용 (각 검색기에 0.5씩 가중치 부여)

각 검색기의 점수에 가중치 0.5를 곱합니다:
```
BM25 검색기:
- 문서 A: 0.9 × 0.5 = 0.45
- 문서 B: 0.8 × 0.5 = 0.40
- 문서 C: 0.7 × 0.5 = 0.35
- 문서 D: 0.6 × 0.5 = 0.30

임베딩 검색기:
- 문서 A: 0.95 × 0.5 = 0.475
- 문서 E: 0.85 × 0.5 = 0.425
- 문서 F: 0.75 × 0.5 = 0.375
- 문서 B: 0.65 × 0.5 = 0.325
```


#### 3. 중복 문서 점수 합산

동일한 문서가 여러 검색기에서 나온 경우, 가중치가 적용된 점수들을 합산합니다:
```
- 문서 A: 0.45 (BM25) + 0.475 (임베딩) = 0.925
- 문서 B: 0.40 (BM25) + 0.325 (임베딩) = 0.725
- 문서 C: 0.35 (BM25만) = 0.35
- 문서 D: 0.30 (BM25만) = 0.30
- 문서 E: 0.425 (임베딩만) = 0.425
- 문서 F: 0.375 (임베딩만) = 0.375
```


#### 4. 점수 기준 정렬 및 최종 결과 반환
```
모든 문서를 최종 점수 기준으로 내림차순 정렬하고, 상위 k개(예: k=4)를 반환합니다:

최종 정렬:
1. 문서 A: 0.925 (두 검색기 모두에서 높은 점수)
2. 문서 B: 0.725 (두 검색기 모두에서 등장)
3. 문서 E: 0.425 (임베딩 검색기에서만)
4. 문서 F: 0.375 (임베딩 검색기에서만)
5. 문서 C: 0.35 (BM25 검색기에서만)
6. 문서 D: 0.30 (BM25 검색기에서만)

최종 반환 결과(k=4): 문서 A, B, E, F
```

### 핵심 포인트
1. **중복 문서 강화**: 두 검색기 모두에서 나온 문서(A, B)는 점수가 합산되어 순위가 높아집니다. 이는 여러 방식으로 관련성이 확인된 문서가 우선시됨을 의미합니다.

2. **다양한 결과 포함**: 각 검색기의 고유한 강점을 활용하여 키워드 매칭(BM25)과 의미적 유사성(임베딩) 모두에서 관련성 높은 문서를 포함합니다.

3. **가중치 영향**: 만약 BM25에 더 높은 가중치(예: 0.7)를 부여한다면, BM25 결과가 최종 순위에 더 큰 영향을 미치게 됩니다.

In [16]:
pip install -U langchain

In [17]:
ensemble_retriever = EnsembleRetriever(
                    retrievers = [bm25_retriever, chroma_retriever], weight = {0.5,0.5})

docs = ensemble_retriever.invoke("삼성전자의 사업 영역은?")

In [18]:
# 문서의 개수는 총 4개이다.
len(docs)

4

## 앙상블 결과

In [19]:
for i in docs:
    print(i.metadata)
    print(":")
    print(i.page_content.replace('\n',' '))
    print(len(i.page_content.replace('\n',' ')))
    print("*"*30)

{'producer': 'itext-paulo-155 (itextpdf.sf.net-lowagie.com)', 'creator': 'nPDF (pdftk 1.41)', 'creationdate': '2017-08-16T00:21:02-08:00', 'moddate': '2017-08-16T00:21:02-08:00', 'source': 'https://wdr.ubion.co.kr/wowpass/img/event/gsat_170823/gsat_170823.pdf', 'total_pages': 27, 'page': 10, 'page_label': '11'}
:
12 Q2 삼성전자의 Harman사 인수에 대해 어떻게 생각하십니까 ? A  삼성전자가 최근 80억달러(약 9조)에 미국의 전장업체(자동차 전자기기에 대한 사업)인 Harman 사를  인수하기로 결정했습니다. 하만사는 세계적으로 자동차용 인포테인먼트와 텔레매틱스 시장에서 점유 율 1~2위를 달리고 있으며 매출도 상당합니다. 이번 삼성전자의 하만 인수는 모바일 및 가전 시장에서  나아가 Connected Car 시장 진입에 진출했다는 것에 큰 의의가 있습니다. 향후 10년 시장 내 Connected  Car의 비중은 90%에 육박할 전망입니다. LG전자 역시 자동차부품(VC) 사업 부문에 있어 상당한 경쟁  위협 요인가 될 것으로 내다봤습니다. <관련기사> 전장사업 강자 하만과 손잡은 삼성…스마트카 ‘티어1’ 노린다 (한국경제. 2016-11-21) 삼성전자는 자동차 전장(電裝)사업에서 후발 주자다. 2005년 전장사업을 시작한 LG전자는
474
******************************
{'total_pages': 27, 'moddate': '2017-08-16T00:21:02-08:00', 'page_label': '10', 'page': 9, 'creationdate': '2017-08-16T00:21:02-08:00', 'source': 'https://wdr.ubion.co.kr/wowp

## 답변 얻기

In [20]:
from langchain.chat_models import ChatOpenAI
from langchain.chains import RetrievalQA

llm_model = ChatOpenAI(model_name='gpt-4o',
                       api_key="여러분의 Key 값",
                       temperature=0)

question = "삼성전자의 사업 영역은?"

<ipython-input-20-2b7494ce6963>:4: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  llm_model = ChatOpenAI(model_name='gpt-4o',


In [21]:
# 앙상블 리트리버
qa = RetrievalQA.from_chain_type(
    llm = llm_model,
    chain_type='stuff',
    retriever = ensemble_retriever)

res = qa(question)
res['result']

<ipython-input-21-57bb9d34aac9>:7: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  res = qa(question)


UnicodeEncodeError: 'ascii' codec can't encode characters in position 7-10: ordinal not in range(128)